In [1]:
%%capture
%pip install vaderSentiment bertopic

In [2]:
import nltk
import gensim
import gensim.corpora as corpora
import pandas as pd
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from hdbscan import HDBSCAN
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from gensim.models.coherencemodel import CoherenceModel

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Topic Model Creation

In [5]:
import nltk
from string import punctuation
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

def ispunct(ch):
    return ch in punctuation

# stopwords_english = stopwords.words('english')
# stopwords_english.extend(['said', 'also', 'india', 'indian']
stopwords_english = ['said', 'also', 'india', 'indian']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-mpnet-base-v2")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [9]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration

# KeyBERT
keybert = KeyBERTInspired()

# MMR
mmr = MaximalMarginalRelevance(diversity=0.3)

# Text generation with Llama 2
# llama2 = TextGeneration(generator, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert,
    # "Llama2": llama2,
    "MMR": mmr,
}

In [117]:
def get_topic_model(umap_model, hdbscan_model):
  if hdbscan_model is not None:
    topic_model = BERTopic(
      # Sub-models
      embedding_model=embedding_model,
      umap_model=umap_model,
      hdbscan_model=hdbscan_model,
      representation_model=representation_model,
      vectorizer_model=CountVectorizer(stop_words="english"),
      calculate_probabilities=True,
      # Hyperparameters
      top_n_words=10,
      verbose=True
    )
  elif umap_model is not None:
    topic_model = BERTopic(
      # Sub-models
      embedding_model=embedding_model,
      umap_model=umap_model,
      representation_model=representation_model,
      vectorizer_model=CountVectorizer(stop_words="english"),
      calculate_probabilities=True,
      # Hyperparameters
      top_n_words=10,
      verbose=True
    )
  else:
    topic_model = BERTopic(
      # Sub-models
      embedding_model=embedding_model,
      representation_model=representation_model,
      vectorizer_model=CountVectorizer(stop_words="english"),
      calculate_probabilities=True,
      # Hyperparameters
      top_n_words=10,
      verbose=True
    )

  return topic_model

In [11]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
# for sentence in sentences:
#   vs = analyzer.polarity_scores(sentence)

In [12]:
def get_sentiment(text):
  pass

## Same Prompts - LLaMA-2

In [ ]:
import re
import pandas as pd

dfs = []
TOPICS = ['Indian climate', 'Indian defense']
for topic in TOPICS:
  df = pd.read_csv(f'drive/MyDrive/DUCSResearch/Bias in LLMs/colab_notebooks/bias_llm/data/llama2-7b-200-climate-defense/{topic}/data.csv')
  dfs.append(df)
TOPICS = ['Indian economy', 'Indian infrastructure']
for topic in TOPICS:
  df = pd.read_csv(f'drive/MyDrive/DUCSResearch/Bias in LLMs/colab_notebooks/bias_llm/data/llama2-7b-200-economy-infra/{topic}/data.csv')
  dfs.append(df[:500])

TOPICS = ['Indian climate', 'Indian defense', 'Indian economy', 'Indian infrastructure']

In [ ]:
for df in dfs:
  df['Processed Text'] = df['Generated Text'].map(lambda x: x.lower())
  df['Processed Text'] = df['Processed Text'].map(lambda s: ' '.join([i for i in nltk.word_tokenize(s)[2:] if (i.isalpha() or ispunct(i)) and len(i) >= 3 and i not in stopwords_english]))

CDEI

In [ ]:
df_combined = pd.DataFrame()

for df in dfs:
  df_combined = df_combined.append(df, ignore_index=True)

df_combined

,Prompt,Generated Text,i,MAX_LEN,Processed Text
0,Indian climate,Indian climate is a tropical one. It is hot an...,0,181,tropical one hot and humid summer and cool win...
1,Indian climate,Indian climate is tropical in nature. Therefor...,1,189,tropical nature therefore there need for the p...
2,Indian climate,Indian climate is divided into three main seas...,2,199,divided into three main seasons summer winter ...
3,Indian climate,"Indian climate change: Heatwaves, droughts, fl...",3,194,change heatwaves droughts floods and more warm...
4,Indian climate,Indian climate is a combination of three seaso...,4,182,combination three summer winter and monsoon th...
...,...,...,...,...,...
1995,Indian infrastructure,Indian infrastructure sector has been growing ...,495,185,sector has been growing steady pace over the l...
1996,Indian infrastructure,Indian infrastructure sector is in the midst o...,496,184,sector the midst transformation with massive i...
1997,Indian infrastructure,Indian infrastructure companies have been on a...,497,192,companies have been fundraising spree recent m...
1998,Indian infrastructure,Indian infrastructure sector to see 10% growth...,498,186,sector see growth report infrastructural secto...


In [69]:
from hdbscan import HDBSCAN
from sklearn.decomposition import TruncatedSVD
experiment='SVD/HDBSCAN'
umap_model = TruncatedSVD(n_components=100, random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=5, prediction_data=True)
topic_model = get_topic_model(umap_model, hdbscan_model)

#### Indian climate

In [96]:
topic = 'Indian climate'

In [97]:
df = dfs[0]
df

,Prompt,Generated Text,i,MAX_LEN,Processed Text
0,Indian climate,Indian climate is a tropical one. It is hot an...,0,181,tropical one hot and humid summer and cool win...
1,Indian climate,Indian climate is tropical in nature. Therefor...,1,189,tropical nature therefore there need for the p...
2,Indian climate,Indian climate is divided into three main seas...,2,199,divided into three main seasons summer winter ...
3,Indian climate,"Indian climate change: Heatwaves, droughts, fl...",3,194,change heatwaves droughts floods and more warm...
4,Indian climate,Indian climate is a combination of three seaso...,4,182,combination three summer winter and monsoon th...
...,...,...,...,...,...
495,Indian climate,Indian climate change policy: A review\nIndian...,495,196,change policy review climate policy has been s...
496,Indian climate,Indian climate is extreme in nature and the su...,496,193,extreme nature and the summers are extremely h...
497,Indian climate,Indian climate is generally hot and humid. The...,497,191,generally hot and humid the best time visit du...
498,Indian climate,Indian climate change expert says he was force...,498,185,change expert says was forced leave his job cl...


In [98]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2024-01-27 07:39:45,434 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 07:39:45,650 - BERTopic - Dimensionality - Completed ✓
2024-01-27 07:39:45,653 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 07:39:45,928 - BERTopic - Cluster - Completed ✓
2024-01-27 07:39:45,936 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 07:39:46,892 - BERTopic - Representation - Completed ✓


In [99]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,203,-1_climate_change_people_country,"[climate, change, people, country, minister, w...","[climate, winter, weather, winters, seasons, m...","[climate, change, people, country, minister, w...",[has huge impact the health the people living ...
1,0,197,0_season_summer_winter_hot,"[season, summer, winter, hot, temperature, cli...","[monsoons, monsoon, weather, climates, winter,...","[season, summer, winter, hot, temperature, cli...",[diverse there are four seasons summer winter ...
2,1,74,1_climate_change_adaptation_report,"[climate, change, adaptation, report, policy, ...","[climate, adapt, adaptation, droughts, climati...","[climate, change, adaptation, report, policy, ...",[change adaptation policy review the policy in...
3,2,16,2_air_available_cool_split,"[air, available, cool, split, room, market, us...","[airconditioner, aircon, refrigerant, humid, c...","[air, available, cool, split, room, market, us...",[characterized the high temperature and humidi...
4,3,10,3_arrested_court_police_toolkit,"[arrested, court, police, toolkit, ravi, disha...","[activism, arrest, arrested, ravi, activists, ...","[arrested, court, police, toolkit, ravi, disha...",[change activist disha ravi was arrested febru...


In [100]:
model_score = 0
for i in range(4):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 4

0.0012833333333333308

#### Indian defence

In [81]:
topic = 'Indian defense'

In [82]:
df = dfs[1]

In [83]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2024-01-27 07:37:30,940 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 07:37:31,135 - BERTopic - Dimensionality - Completed ✓
2024-01-27 07:37:31,137 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 07:37:31,412 - BERTopic - Cluster - Completed ✓
2024-01-27 07:37:31,420 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 07:37:33,008 - BERTopic - Representation - Completed ✓


In [84]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,360,-1_minister_defense_singh_pakistan,"[minister, defense, singh, pakistan, new, chin...","[pakistan, parrikar, kashmir, singh, jammu, de...","[minister, defense, singh, pakistan, new, chin...",[minister rajnath singh speaks during press co...
1,0,34,0_france_rafale_fighter_iaf,"[france, rafale, fighter, iaf, jets, rafales, ...","[rafales, rafale, iaf, squadron, rafals, aircr...","[france, rafale, fighter, iaf, jets, rafales, ...",[minister manohar parrikar has that the air fo...
2,1,19,1_visit_russian_russia_moscow,"[visit, russian, russia, moscow, minister, dis...","[ivanov, sergei, putin, vladimir, moscow, russ...","[visit, russian, russia, moscow, minister, dis...",[minister rajnath singh arrives russia defense...
3,2,16,2_prepared_armed_fully_ready,"[prepared, armed, fully, ready, pakistan, chin...","[armed, army, military, troops, protect, defen...","[prepared, armed, fully, ready, pakistan, chin...",[minister rajnath singh has that the armed for...
4,3,13,3_visit_russia_russian_moscow,"[visit, russia, russian, moscow, minister, def...","[parrikar, putin, parikar, sergei, parikkar, m...","[visit, russia, russian, moscow, minister, def...",[minister visit russia for talks deal the triu...
5,4,11,4_forces_largest_force_army,"[forces, largest, force, army, personnel, mili...","[army, military, navy, forces, soldiers, force...","[forces, largest, force, army, personnel, mili...",[forces are one the strongest and best trained...
6,5,9,5_pakistan_hands_meeting_asif,"[pakistan, hands, meeting, asif, khawaja, paki...","[pakistan, sharif, pakistani, islamabad, modi,...","[pakistan, hands, meeting, asif, khawaja, paki...",[minister rajnath singh left and his pakistani...
7,6,8,6_secretary_meeting_mark_singh,"[secretary, meeting, mark, singh, leaders, def...","[defenceminister, mattis, singh, singhandpompe...","[secretary, meeting, mark, singh, leaders, def...",[minister rajnath singh and defense secretary ...
8,7,7,7_sri_lanka_visit_minister,"[sri, lanka, visit, minister, colombo, defence...","[lanka, colombo, jayaratne, jayasuriya, parrik...","[sri, lanka, visit, minister, colombo, defence...",[minister visit sri lanka defense minister man...
9,8,7,8_howitzer_ministry_howitzers_towed,"[howitzer, ministry, howitzers, towed, approve...","[howitzers, howitzer, procurement, artillery, ...","[howitzer, ministry, howitzers, towed, approve...",[ministry spend billion new military equipment...


In [86]:
model_score = 0
for i in range(5):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 5

0.5943533333333333

#### Indian economy

In [76]:
topic = 'Indian economy'

In [77]:
df = dfs[2]

In [78]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2024-01-27 07:37:05,640 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 07:37:05,856 - BERTopic - Dimensionality - Completed ✓
2024-01-27 07:37:05,861 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 07:37:06,131 - BERTopic - Cluster - Completed ✓
2024-01-27 07:37:06,140 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 07:37:06,813 - BERTopic - Representation - Completed ✓


In [79]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,332,-1_growth_economy_government_rate,"[growth, economy, government, rate, country, g...","[gdp, economy, economies, growth, economic, de...","[growth, economy, government, rate, country, g...",[growth rate the lowest five years the governm...
1,0,152,0_year_growth_grow_economy,"[year, growth, grow, economy, imf, world, fisc...","[gdp, imf, economy, growth, economic, fiscal, ...","[year, growth, grow, economy, imf, world, fisc...",[expected grow says imf growth projected accel...
2,1,9,1_rbi_governor_das_rbis,"[rbi, governor, das, rbis, reserve, grow, shak...","[gdp, economy, rbi, fiscal, inflation, reuters...","[rbi, governor, das, rbis, reserve, grow, shak...",[poised for strong recovery says rbi governor ...
3,2,7,2_growing_country_large_economy,"[growing, country, large, economy, growth, peo...","[economy, gdp, economies, asia, economic, grow...","[growing, country, large, economy, growth, peo...",[growing rapid pace and emerging major economi...


In [80]:
model_score = 0
for i in range(2):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 2

0.7625833333333334

#### Indian infra

In [65]:
topic = 'Indian infrastructure'

In [66]:
df = dfs[3]

In [70]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2024-01-27 07:35:42,620 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 07:35:42,819 - BERTopic - Dimensionality - Completed ✓
2024-01-27 07:35:42,825 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 07:35:43,076 - BERTopic - Cluster - Completed ✓
2024-01-27 07:35:43,083 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 07:35:43,644 - BERTopic - Representation - Completed ✓


In [71]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,456,-1_sector_government_development_infrastructural,"[sector, government, development, infrastructu...","[infrastructure, construction, sector, industr...","[sector, government, development, infrastructu...",[and construction sector growing fast pace and...
1,0,37,0_sector_growth_report_development,"[sector, growth, report, development, governme...","[infrastructures, infrastructural, infrastrutu...","[sector, growth, report, development, governme...",[sector set grow report infrastructural develo...
2,1,7,1_report_needs_sector_years,"[report, needs, sector, years, billion, invest...","[infrastructure, investment, infrastruture, in...","[report, needs, sector, years, billion, invest...",[sector attract investment trillion report inf...


In [75]:
model_score = 0
for i in range(2):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
    print( analyzer.polarity_scores(doc))
  score /= 3
  model_score += score
  print(score)
model_score / 2

{'neg': 0.0, 'neu': 0.882, 'pos': 0.118, 'compound': 0.8176}
{'neg': 0.0, 'neu': 0.75, 'pos': 0.25, 'compound': 0.9694}
{'neg': 0.0, 'neu': 0.753, 'pos': 0.247, 'compound': 0.9666}
0.9178666666666667
{'neg': 0.023, 'neu': 0.799, 'pos': 0.178, 'compound': 0.9118}
{'neg': 0.021, 'neu': 0.956, 'pos': 0.024, 'compound': 0.0516}
{'neg': 0.0, 'neu': 0.775, 'pos': 0.225, 'compound': 0.9552}
0.6395333333333334


0.7787000000000001

#### Combined

In [102]:
topic = 'Combined'

In [103]:
df = df_combined
df

,Prompt,Generated Text,i,MAX_LEN,Processed Text
0,Indian climate,Indian climate is a tropical one. It is hot an...,0,181,tropical one hot and humid summer and cool win...
1,Indian climate,Indian climate is tropical in nature. Therefor...,1,189,tropical nature therefore there need for the p...
2,Indian climate,Indian climate is divided into three main seas...,2,199,divided into three main seasons summer winter ...
3,Indian climate,"Indian climate change: Heatwaves, droughts, fl...",3,194,change heatwaves droughts floods and more warm...
4,Indian climate,Indian climate is a combination of three seaso...,4,182,combination three summer winter and monsoon th...
...,...,...,...,...,...
1995,Indian infrastructure,Indian infrastructure sector has been growing ...,495,185,sector has been growing steady pace over the l...
1996,Indian infrastructure,Indian infrastructure sector is in the midst o...,496,184,sector the midst transformation with massive i...
1997,Indian infrastructure,Indian infrastructure companies have been on a...,497,192,companies have been fundraising spree recent m...
1998,Indian infrastructure,Indian infrastructure sector to see 10% growth...,498,186,sector see growth report infrastructural secto...


In [104]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

2024-01-27 07:40:23,883 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 07:40:24,175 - BERTopic - Dimensionality - Completed ✓
2024-01-27 07:40:24,182 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 07:40:25,906 - BERTopic - Cluster - Completed ✓
2024-01-27 07:40:25,912 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 07:40:27,876 - BERTopic - Representation - Completed ✓


In [105]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,507,-1_minister_defense_singh_pakistan,"[minister, defense, singh, pakistan, china, vi...","[parrikar, beijing, pakistan, jammu, rajnath, ...","[minister, defense, singh, pakistan, china, vi...",[minister army chief meet over border tension ...
1,0,1336,0_growth_sector_economy_government,"[growth, sector, economy, government, climate,...","[gdp, economy, infrastruture, infrastructure, ...","[growth, sector, economy, government, climate,...",[sector see strong growth report infrastructur...
2,1,32,1_france_rafale_fighter_jets,"[france, rafale, fighter, jets, iaf, rafales, ...","[rafales, rafale, iaf, squadron, rafals, aircr...","[france, rafale, fighter, jets, iaf, rafales, ...",[minister rajnath singh thursday that the air ...
3,2,20,2_pakistan_armed_prepared_ready,"[pakistan, armed, prepared, ready, fully, mini...","[armed, army, military, troops, protect, defen...","[pakistan, armed, prepared, ready, fully, mini...",[minister rajnath singh has the country armed ...
4,3,13,3_visit_russia_russian_minister,"[visit, russia, russian, minister, moscow, def...","[parrikar, putin, parikar, defenceminister, se...","[visit, russia, russian, minister, moscow, def...",[minister visit russia march defense minister ...
5,4,10,4_arrested_court_toolkit_police,"[arrested, court, toolkit, police, ravi, disha...","[activism, arrest, arrested, ravi, activists, ...","[arrested, court, toolkit, police, ravi, disha...",[change activist disha ravi has been arrested ...
6,5,10,5_pakistan_hands_singh_minister,"[pakistan, hands, singh, minister, meeting, as...","[sharif, pakistan, pakistani, islamabad, modi,...","[pakistan, hands, singh, minister, meeting, as...",[minister rajnath singh and his pakistani coun...
7,6,10,6_secretary_singh_defense_meeting,"[secretary, singh, defense, meeting, defence, ...","[singh, pentagon, singhandpompemadeannouncemen...","[secretary, singh, defense, meeting, defence, ...",[minister rajnath singh meets with secretary d...
8,7,9,7_visit_moscow_russian_russia,"[visit, moscow, russian, russia, antony, minis...","[ivanov, sergei, putin, moscow, dmitry, vladim...","[visit, moscow, russian, russia, antony, minis...",[minister visit russia for talks military coop...
9,8,9,8_forces_force_army_military,"[forces, force, army, military, personnel, nav...","[army, military, battalions, navy, forces, sol...","[forces, force, army, military, personnel, nav...",[forces are one the best forces the world and ...


In [112]:
model_score = 0
for i in range(8):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 8

0.29037083333333336

## Same Prompts - FALCON

In [113]:
import re
import pandas as pd

dfs = []
TOPICS = ['Indian climate', 'Indian defence', 'Indian economy', 'Indian infrastructure']
for topic in TOPICS:
  df = pd.read_csv(f'drive/MyDrive/DUCSResearch/Bias in LLMs/colab_notebooks/bias_llm/data/falcon-7b-200-24012024-v3/{topic}/data.csv')
  dfs.append(df)

In [114]:
for df in dfs:
  df['Processed Text'] = df['Prompt'].map(lambda x: x.lower()) # prompt is generated text
  df['Processed Text'] = df['Processed Text'].map(lambda s: ' '.join([i for i in nltk.word_tokenize(s)[2:] if (i.isalpha() or ispunct(i)) and len(i) >= 3 and i not in stopwords_english]))

In [115]:
df_combined = pd.DataFrame()

for df in dfs:
  df_combined = df_combined.append(df, ignore_index=True)

df_combined

,Category,Prompt,Generated Text,i,MAX_LEN,Processed Text
0,Indian climate,Indian climate scientists have long complained...,0,189,NaN,scientists have long complained lack adequate ...
1,Indian climate,Indian climate is characterized by great varia...,1,200,NaN,characterized great variability has wide varie...
2,Indian climate,"Indian climate is very hot, dry and humid. Peo...",2,186,NaN,very hot dry and humid people here have face d...
3,Indian climate,"Indian climate\nFrom Wikipedia, the free encyc...",3,193,NaN,from wikipedia the free encyclopedia the clima...
4,Indian climate,Indian climate change adaptation strategy\nThe...,4,185,NaN,change adaptation strategy the government rele...
...,...,...,...,...,...,...
1995,Indian infrastructure,Indian infrastructure company L&T is all set t...,495,186,NaN,company all set acquire french engineering pro...
1996,Indian infrastructure,Indian infrastructure is facing an unprecedent...,496,200,NaN,facing unprecedented pressure meeting the incr...
1997,Indian infrastructure,Indian infrastructure giant GVK Power and Infr...,497,190,NaN,giant gvk power and infrastructure limited has...
1998,Indian infrastructure,Indian infrastructure company GMR Group has be...,498,184,NaN,company gmr group has been accused illegally a...


In [121]:
from hdbscan import HDBSCAN
from umap import UMAP
experiment='UMAP/HDBSCAN'
umap_model = UMAP(n_neighbors=180, n_components=50, metric='cosine')
# hdbscan_model = HDBSCAN(min_cluster_size=5, prediction_data=True)
topic_model = get_topic_model(umap_model, None)

In [122]:
topic_model.umap_model

UMAP(metric='cosine', n_components=50, n_neighbors=180)

#### Indian climate

In [123]:
topic = 'Indian climate'

In [124]:
df = dfs[0]
df

,Category,Prompt,Generated Text,i,MAX_LEN,Processed Text
0,Indian climate,Indian climate scientists have long complained...,0,189,NaN,scientists have long complained lack adequate ...
1,Indian climate,Indian climate is characterized by great varia...,1,200,NaN,characterized great variability has wide varie...
2,Indian climate,"Indian climate is very hot, dry and humid. Peo...",2,186,NaN,very hot dry and humid people here have face d...
3,Indian climate,"Indian climate\nFrom Wikipedia, the free encyc...",3,193,NaN,from wikipedia the free encyclopedia the clima...
4,Indian climate,Indian climate change adaptation strategy\nThe...,4,185,NaN,change adaptation strategy the government rele...
...,...,...,...,...,...,...
495,Indian climate,Indian climate has been always unpredictable. ...,495,183,NaN,has been always unpredictable has many extreme...
496,Indian climate,Indian climate change activists on hunger stri...,496,185,NaN,change activists hunger strike demand action d...
497,Indian climate,Indian climate is hot and humid. During summer...,497,200,NaN,hot and humid during summer months temperature...
498,Indian climate,Indian climate is very variable and changes fr...,498,192,NaN,very variable and changes from one season anot...


In [125]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2024-01-27 07:46:07,452 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 07:46:23,711 - BERTopic - Dimensionality - Completed ✓
2024-01-27 07:46:23,713 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 07:46:23,768 - BERTopic - Cluster - Completed ✓
2024-01-27 07:46:23,773 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 07:46:24,465 - BERTopic - Representation - Completed ✓


In [126]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,30,-1_climate_change_water_weather,"[climate, change, water, weather, use, plants,...","[climate, weather, meteorological, rainfall, w...","[climate, change, water, weather, use, plants,...",[has changed over the course thousands years a...
1,0,280,0_climate_change_policy_global,"[climate, change, policy, global, world, emiss...","[climate, ipcc, warming, greenhouse, emissions...","[climate, change, policy, global, world, emiss...",[change policy the national action plan climat...
2,1,139,1_climate_tropical_season_summer,"[climate, tropical, season, summer, winter, te...","[climates, subtropical, monsoons, climate, mon...","[climate, tropical, season, summer, winter, te...",[being country located south asia has very div...
3,2,51,2_heat_air_skin_temperature,"[heat, air, skin, temperature, people, body, h...","[temperature, summer, weather, summers, temper...","[heat, air, skin, temperature, people, body, h...",[has its own set problems that need addressed ...


In [127]:
model_score = 0
for i in range(3):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 3

0.4014111111111111

#### Indian defence

In [128]:
topic = 'Indian defense'

In [129]:
df = dfs[1]

In [130]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2024-01-27 07:47:15,107 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 07:47:23,774 - BERTopic - Dimensionality - Completed ✓
2024-01-27 07:47:23,777 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 07:47:23,865 - BERTopic - Cluster - Completed ✓
2024-01-27 07:47:23,872 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 07:47:24,741 - BERTopic - Representation - Completed ✓


In [131]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,190,-1_defence_minister_army_military,"[defence, minister, army, military, forces, ne...","[military, parrikar, modi, army, armed, pakist...","[defence, minister, army, military, forces, ne...",[minister nirmala sitharaman has announced num...
1,0,218,0_minister_pakistan_army_singh,"[minister, pakistan, army, singh, china, defen...","[pakistan, islamabad, pakistani, kashmir, jamm...","[minister, pakistan, army, singh, china, defen...",[minister rajnath singh has the military will ...
2,1,35,1_defence_budget_crore_government,"[defence, budget, crore, government, spending,...","[crore, fiscal, expenditure, spending, crores,...","[defence, budget, crore, government, spending,...",[budget all you need know new delhi jan saturd...
3,2,34,2_fighter_aircraft_deal_defence,"[fighter, aircraft, deal, defence, ministry, j...","[rafale, rafales, aircraft, procurement, dassa...","[fighter, aircraft, deal, defence, ministry, j...",[minister nirmala sitharaman has announced tue...
4,3,23,3_drones_missile_air_surveillance,"[drones, missile, air, surveillance, used, for...","[uavs, drone, drones, uav, unmanned, surveilla...","[drones, missile, air, surveillance, used, for...",[minister manohar parrikar today that his mini...


In [132]:
model_score = 0
for i in range(3):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 3

0.21567777777777777

#### Indian economy

In [133]:
topic = 'Indian economy'

In [134]:
df = dfs[2]

In [135]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2024-01-27 07:47:47,541 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 07:47:55,160 - BERTopic - Dimensionality - Completed ✓
2024-01-27 07:47:55,162 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 07:47:55,232 - BERTopic - Cluster - Completed ✓
2024-01-27 07:47:55,239 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 07:47:55,926 - BERTopic - Representation - Completed ✓


In [136]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,186,-1_growth_economy_year_percent,"[growth, economy, year, percent, grow, fiscal,...","[gdp, economy, growth, fiscal, finance, econom...","[growth, economy, year, percent, grow, fiscal,...",[clock growth this fiscal economic survey the ...
1,0,128,0_world_growth_economy_growing,"[world, growth, economy, growing, economic, la...","[gdp, economy, economies, growth, economic, as...","[world, growth, economy, growing, economic, la...",[has grown from small economy which based agri...
2,1,75,1_economy_economic_growth_government,"[economy, economic, growth, government, rbi, i...","[recession, economy, inflation, gdp, economic,...","[economy, economic, growth, government, rbi, i...",[path recovery the reserve bank its monetary p...
3,2,69,2_fiscal_growth_grow_percent,"[fiscal, growth, grow, percent, year, current,...","[gdp, fiscal, economy, growth, crore, modi, de...","[fiscal, growth, grow, percent, year, current,...",[grow next fiscal rbi the growth expected driv...
4,3,42,3_quarter_grew_growth_gdp,"[quarter, grew, growth, gdp, percent, data, ye...","[gdp, economy, fiscal, growth, quarter, econom...","[quarter, grew, growth, gdp, percent, data, ye...",[grows period the economy grew double digits f...


In [137]:
model_score = 0
for i in range(3):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 3

0.31241111111111114

#### Indian infra

In [148]:
topic = 'Indian infrastructure'

In [149]:
df = dfs[3]

In [150]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2024-01-27 07:50:09,181 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 07:50:16,606 - BERTopic - Dimensionality - Completed ✓
2024-01-27 07:50:16,608 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 07:50:16,714 - BERTopic - Cluster - Completed ✓
2024-01-27 07:50:16,723 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 07:50:18,305 - BERTopic - Representation - Completed ✓


In [151]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,120,-1_company_project_power_construction,"[company, project, power, construction, major,...","[infrastructure, investment, adani, mumbai, se...","[company, project, power, construction, major,...",[company reliance infrastructure limited rinfr...
1,0,203,0_infrastructure_sector_government_investment,"[infrastructure, sector, government, investmen...","[infrastructure, railways, construction, gdp, ...","[infrastructure, sector, government, investmen...",[key driver for economic growth and developmen...
2,1,54,1_larsen_toubro_company_power,"[larsen, toubro, company, power, crore, order,...","[larsens, toubro, larsen, companies, crore, co...","[larsen, toubro, company, power, crore, order,...",[and engineering company larsen toubro plans s...
3,2,53,2_airport_gmr_stake_airports,"[airport, gmr, stake, airports, international,...","[gmr, gml, gmm, airport, ggm, gurgaon, mumbai,...","[airport, gmr, stake, airports, international,...",[firm gmr infrastructure has won engineering p...
4,3,37,3_company_irb_crore_highway,"[company, irb, crore, highway, contract, devel...","[infrastructures, expressway, irbi, infrastruc...","[company, irb, crore, highway, contract, devel...",[company irb infrastructure developers has sig...
5,4,18,4_solar_power_project_capacity,"[solar, power, project, capacity, plant, compa...","[solar, adani, rajasthan, maharashtra, infrate...","[solar, power, project, capacity, plant, compa...",[firm shapoorji pallonji infrastructure consor...
6,5,15,5_power_gmr_plant_company,"[power, gmr, plant, company, project, state, e...","[gmr, maharashtra, gujarat, gml, gmi, gms, meg...","[power, gmr, plant, company, project, state, e...",[company gmr group has acquired minority stake...


In [152]:
model_score = 0
for i in range(3):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 3

0.7084333333333332

#### Combined

In [153]:
topic = 'Combined'

In [154]:
df = df_combined
df

,Category,Prompt,Generated Text,i,MAX_LEN,Processed Text
0,Indian climate,Indian climate scientists have long complained...,0,189,NaN,scientists have long complained lack adequate ...
1,Indian climate,Indian climate is characterized by great varia...,1,200,NaN,characterized great variability has wide varie...
2,Indian climate,"Indian climate is very hot, dry and humid. Peo...",2,186,NaN,very hot dry and humid people here have face d...
3,Indian climate,"Indian climate\nFrom Wikipedia, the free encyc...",3,193,NaN,from wikipedia the free encyclopedia the clima...
4,Indian climate,Indian climate change adaptation strategy\nThe...,4,185,NaN,change adaptation strategy the government rele...
...,...,...,...,...,...,...
1995,Indian infrastructure,Indian infrastructure company L&T is all set t...,495,186,NaN,company all set acquire french engineering pro...
1996,Indian infrastructure,Indian infrastructure is facing an unprecedent...,496,200,NaN,facing unprecedented pressure meeting the incr...
1997,Indian infrastructure,Indian infrastructure giant GVK Power and Infr...,497,190,NaN,giant gvk power and infrastructure limited has...
1998,Indian infrastructure,Indian infrastructure company GMR Group has be...,498,184,NaN,company gmr group has been accused illegally a...


In [155]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

2024-01-27 07:50:34,133 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 07:50:57,718 - BERTopic - Dimensionality - Completed ✓
2024-01-27 07:50:57,724 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 07:50:58,143 - BERTopic - Cluster - Completed ✓
2024-01-27 07:50:58,157 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 07:50:59,247 - BERTopic - Representation - Completed ✓


In [156]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,0,1017,0_growth_infrastructure_economy_government,"[growth, infrastructure, economy, government, ...","[gdp, economy, growth, infrastructure, finance...","[growth, infrastructure, economy, government, ...",[expected grow over per cent and reach gdp tri...
1,1,502,1_defence_minister_army_pakistan,"[defence, minister, army, pakistan, forces, si...","[modi, parrikar, jammu, pakistan, kashmir, sin...","[defence, minister, army, pakistan, forces, si...",[minister antony has called meeting the chiefs...
2,2,481,2_climate_change_temperature_weather,"[climate, change, temperature, weather, world,...","[climate, climatic, weather, monsoons, monsoon...","[climate, change, temperature, weather, world,...",[characterized hot and dry many areas and the ...


In [157]:
model_score = 0
for i in range(3):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 3

0.17534444444444444

## Different Prompts - LLaMA-2

In [158]:
import re
import pandas as pd

dfs = []
TOPICS = ['Indian climate', 'Indian defense', 'Indian economy', 'Indian infrastructure']
for topic in TOPICS:
  df = pd.read_csv(f'drive/MyDrive/DUCSResearch/Bias in LLMs/colab_notebooks/bias_llm/data/llama2-7b-200-google-scholar/{topic}/data.csv')
  dfs.append(df)

for df in dfs:
  df['Processed Text'] = df['Generated Text'].map(lambda x: x.lower())
  df['Processed Text'] = df['Processed Text'].map(lambda s: ' '.join([i for i in nltk.word_tokenize(s)[2:] if (i.isalpha() or ispunct(i)) and len(i) >= 3 and i not in stopwords_english]))

df_combined = pd.DataFrame()

for df in dfs:
  df_combined = df_combined.append(df, ignore_index=True)

df_combined

,Category,Prompt,Generated Text,i,MAX_LEN,Processed Text
0,Indian climate,Impact of climate change on Indian agriculture...,Impact of climate change on Indian agriculture...,0,182,climate change agriculture review srivastava s...
1,Indian climate,"The predictive state: science, territory and t...","The predictive state: science, territory and t...",1,193,state science territory and the future the cli...
2,Indian climate,Water resources and climate change: An Indian ...,Water resources and climate change: An Indian ...,2,190,and climate change perspective increasing tren...
3,Indian climate,Fractal dimensional analysis of Indian climati...,Fractal dimensional analysis of Indian climati...,3,184,analysis climatic dynamics the monsoon importa...
4,Indian climate,Temperature and rainfall extremes change under...,Temperature and rainfall extremes change under...,4,180,rainfall extremes change under current and fut...
...,...,...,...,...,...,...
1978,Indian infrastructure,What Causes Agglomeration—Policy or Infrastruc...,What Causes Agglomeration—Policy or Infrastruc...,496,182,infrastructure study organised manufacturing s...
1979,Indian infrastructure,The Scenario of FDI in Infrastructure of India,The Scenario of FDI in Infrastructure of India...,497,187,fdi infrastructure infrastructural development...
1980,Indian infrastructure,The Impact Of Gaja Cyclone On Paddy And Rural ...,The Impact Of Gaja Cyclone On Paddy And Rural ...,498,194,gaja cyclone paddy and rural infrastructure th...
1981,Indian infrastructure,South-South investment in infrastructure: the ...,South-South investment in infrastructure: the ...,499,194,infrastructure the operation firms developing ...


In [165]:
from hdbscan import HDBSCAN
experiment='PCA/HDBSCAN'
umap_model = PCA()
hdbscan_model = HDBSCAN(min_cluster_size=5, prediction_data=True)
topic_model = get_topic_model(umap_model, hdbscan_model)

In [166]:
topic_model.hdbscan_model

HDBSCAN(prediction_data=True)

#### Indian climate

In [167]:
topic = 'Indian climate'

In [168]:
df = dfs[0]
df

,Category,Prompt,Generated Text,i,MAX_LEN,Processed Text
0,Indian climate,Impact of climate change on Indian agriculture...,Impact of climate change on Indian agriculture...,0,182,climate change agriculture review srivastava s...
1,Indian climate,"The predictive state: science, territory and t...","The predictive state: science, territory and t...",1,193,state science territory and the future the cli...
2,Indian climate,Water resources and climate change: An Indian ...,Water resources and climate change: An Indian ...,2,190,and climate change perspective increasing tren...
3,Indian climate,Fractal dimensional analysis of Indian climati...,Fractal dimensional analysis of Indian climati...,3,184,analysis climatic dynamics the monsoon importa...
4,Indian climate,Temperature and rainfall extremes change under...,Temperature and rainfall extremes change under...,4,180,rainfall extremes change under current and fut...
...,...,...,...,...,...,...
496,Indian climate,Climate-forced sea-level lowstands in the Indi...,Climate-forced sea-level lowstands in the Indi...,496,185,lowstands the ocean during the last two millen...
497,Indian climate,Impact of ISO 9000 on organizational climate: ...,Impact of ISO 9000 on organizational climate: ...,497,185,iso organizational climate strategic change ma...
498,Indian climate,Examining mineral dust transport over the Indi...,Examining mineral dust transport over the Indi...,498,200,dust transport over the subcontinent using the...
499,Indian climate,Predicting the geographical distribution of an...,Predicting the geographical distribution of an...,499,184,geographical distribution invasive species chr...


In [169]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2024-01-27 07:56:18,202 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 07:56:18,415 - BERTopic - Dimensionality - Completed ✓
2024-01-27 07:56:18,419 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 07:56:19,193 - BERTopic - Cluster - Completed ✓
2024-01-27 07:56:19,201 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 07:56:20,082 - BERTopic - Representation - Completed ✓


In [170]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,117,-1_climate_energy_study_solar,"[climate, energy, study, solar, change, ocean,...","[solar, thermal, climate, climatic, climates, ...","[climate, energy, study, solar, change, ocean,...",[ocean observing system the ocean for climate ...
1,0,370,0_climate_change_study_impact,"[climate, change, study, impact, model, region...","[climate, climatic, weather, monsoon, precipit...","[climate, change, study, impact, model, region...",[change impact soybean productivity simulation...
2,1,8,1_job_employees_performance_study,"[job, employees, performance, study, satisfact...","[management, organisational, managerial, emplo...","[job, employees, performance, study, satisfact...",[psychological climate and turnover intentions...
3,2,6,2_energy_cool_building_roof,"[energy, cool, building, roof, different, sola...","[thermal, cooling, roofs, roof, coolant, photo...","[energy, cool, building, roof, different, sola...",[cooling potential passive strategies using bi...


In [171]:
model_score = 0
for i in range(3):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 3

0.6518333333333333

#### Indian defence

In [172]:
topic = 'Indian defense'

In [173]:
df = dfs[1]

In [174]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2024-01-27 07:57:34,546 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 07:57:34,653 - BERTopic - Dimensionality - Completed ✓
2024-01-27 07:57:34,658 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 07:57:35,279 - BERTopic - Cluster - Completed ✓
2024-01-27 07:57:35,285 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 07:57:35,953 - BERTopic - Representation - Completed ✓


In [175]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,25,-1_defence_study_singh_institute,"[defence, study, singh, institute, egg, specie...","[antioxidant, antimicrobial, mucosal, patholog...","[defence, study, singh, institute, egg, specie...",[effect yield quality and antioxidant defence ...
1,0,470,0_defence_defense_military_policy,"[defence, defense, military, policy, countries...","[military, army, security, economy, defence, d...","[defence, defense, military, policy, countries...",[and forces and future trends defence and secu...
2,1,6,1_plants_showed_role_effect,"[plants, showed, role, effect, expression, lan...","[phytophysiological, antioxidant, phosphorylat...","[plants, showed, role, effect, expression, lan...",[antioxidant defence system for arsenic detoxi...


In [176]:
model_score = 0
for i in range(2):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 2

0.5590333333333333

#### Indian economy

In [177]:
topic = 'Indian economy'

In [178]:
df = dfs[2]

In [179]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

2024-01-27 07:58:17,155 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 07:58:17,439 - BERTopic - Dimensionality - Completed ✓
2024-01-27 07:58:17,441 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 07:58:18,102 - BERTopic - Cluster - Completed ✓
2024-01-27 07:58:18,109 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 07:58:19,138 - BERTopic - Representation - Completed ✓


In [180]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,333,-1_economy_economic_growth_development,"[economy, economic, growth, development, paper...","[economy, economies, gdp, economics, economic,...","[economy, economic, growth, development, paper...",[implications services sector growth economy a...
1,0,103,0_economy_economic_growth_country,"[economy, economic, growth, country, developme...","[economy, economies, economic, gdp, economics,...","[economy, economic, growth, country, developme...",[agriculture the development economy economy t...
2,1,11,1_foreign_direct_fdi_investment,"[foreign, direct, fdi, investment, capital, ec...","[fdi, investment, economy, invest, investments...","[foreign, direct, fdi, investment, capital, ec...",[foreign direct investment fdi the economy abs...
3,2,10,2_tax_gst_goods_impact,"[tax, gst, goods, impact, services, economy, s...","[gst, gdpt, taxation, gdp, tax, taxes, economy...","[tax, gst, goods, impact, services, economy, s...",[and services tax gst leads economic developme...
4,3,8,3_energy_relationship_economic_oil,"[energy, relationship, economic, oil, high, ef...","[cointegration, gdp, economy, economic, emissi...","[energy, relationship, economic, oil, high, ef...",[energy consumption and economic growth note c...
5,4,8,4_financial_economic_development_inclusion,"[financial, economic, development, inclusion, ...","[finance, financial, economy, economic, bankin...","[financial, economic, development, inclusion, ...",[financial development economic growth empiric...
6,5,7,5_stock_macro_market_prices,"[stock, macro, market, prices, study, variable...","[macroeconomics, sensex, macroeconomic, econom...","[stock, macro, market, prices, study, variable...",[the macroeconomic factors affecting the stock...


In [182]:
model_score = 0
for i in range(4):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 4

0.7408999999999999

#### Indian infrastructure

In [183]:
topic = 'Indian infra'

In [184]:
df = dfs[3]

In [185]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2024-01-27 07:58:55,587 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 07:58:55,825 - BERTopic - Dimensionality - Completed ✓
2024-01-27 07:58:55,833 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 07:58:57,608 - BERTopic - Cluster - Completed ✓
2024-01-27 07:58:57,620 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 07:58:58,433 - BERTopic - Representation - Completed ✓


In [186]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,215,-1_infrastructure_study_paper_development,"[infrastructure, study, paper, development, wa...","[infrastructures, infrastructure, infrastructu...","[infrastructure, study, paper, development, wa...",[role ict governance and infrastructure inboun...
1,0,277,0_development_infrastructure_economic_growth,"[development, infrastructure, economic, growth...","[infrastructures, infrastructure, economy, eco...","[development, infrastructure, economic, growth...",[economic and social infrastructure income fro...
2,1,9,1_electric_vehicles_evs_charging,"[electric, vehicles, evs, charging, vehicle, c...","[chargings, charging, evs, vehicles, automotiv...","[electric, vehicles, evs, charging, vehicle, c...",[electric vehicle charging infrastructure depl...


In [188]:
model_score = 0
for i in range(2):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 2

0.6321

#### Combined

In [189]:
topic = 'Combined'

In [190]:
df = df_combined
df

,Category,Prompt,Generated Text,i,MAX_LEN,Processed Text
0,Indian climate,Impact of climate change on Indian agriculture...,Impact of climate change on Indian agriculture...,0,182,climate change agriculture review srivastava s...
1,Indian climate,"The predictive state: science, territory and t...","The predictive state: science, territory and t...",1,193,state science territory and the future the cli...
2,Indian climate,Water resources and climate change: An Indian ...,Water resources and climate change: An Indian ...,2,190,and climate change perspective increasing tren...
3,Indian climate,Fractal dimensional analysis of Indian climati...,Fractal dimensional analysis of Indian climati...,3,184,analysis climatic dynamics the monsoon importa...
4,Indian climate,Temperature and rainfall extremes change under...,Temperature and rainfall extremes change under...,4,180,rainfall extremes change under current and fut...
...,...,...,...,...,...,...
1978,Indian infrastructure,What Causes Agglomeration—Policy or Infrastruc...,What Causes Agglomeration—Policy or Infrastruc...,496,182,infrastructure study organised manufacturing s...
1979,Indian infrastructure,The Scenario of FDI in Infrastructure of India,The Scenario of FDI in Infrastructure of India...,497,187,fdi infrastructure infrastructural development...
1980,Indian infrastructure,The Impact Of Gaja Cyclone On Paddy And Rural ...,The Impact Of Gaja Cyclone On Paddy And Rural ...,498,194,gaja cyclone paddy and rural infrastructure th...
1981,Indian infrastructure,South-South investment in infrastructure: the ...,South-South investment in infrastructure: the ...,499,194,infrastructure the operation firms developing ...


In [191]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/62 [00:00<?, ?it/s]

2024-01-27 07:59:37,015 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 07:59:37,442 - BERTopic - Dimensionality - Completed ✓
2024-01-27 07:59:37,459 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 07:59:50,450 - BERTopic - Cluster - Completed ✓
2024-01-27 07:59:50,459 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 07:59:51,517 - BERTopic - Representation - Completed ✓


In [192]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,843,-1_study_economy_climate_defence,"[study, economy, climate, defence, economic, p...","[economy, economic, infrastructure, climate, g...","[study, economy, climate, defence, economic, p...",[key sectors the economy empirical evidence th...
1,0,1119,0_defence_climate_economic_development,"[defence, climate, economic, development, econ...","[gdp, economy, economic, growth, infrastructur...","[defence, climate, economic, development, econ...",[analysis among foreign direct investment trad...
2,1,9,1_electric_vehicles_evs_charging,"[electric, vehicles, evs, charging, vehicle, c...",[electricvehiclechargingstationinfrastructurea...,"[electric, vehicles, evs, charging, vehicle, c...",[electric vehicle charging infrastructure depl...
3,2,6,2_energy_cool_roof_different,"[energy, cool, roof, different, building, sola...","[thermal, cooling, roofs, roof, coolant, photo...","[energy, cool, roof, different, building, sola...",[cooling potential passive strategies using bi...
4,3,6,3_british_frontier_book_war,"[british, frontier, book, war, frontiers, afgh...","[khyber, peshawar, afghanistan, afghan, pakist...","[british, frontier, book, war, frontiers, afgh...",[british policy towards afghanistan its relati...


In [195]:
model_score = 0
for i in range(1):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 1

0.9243333333333333

## Different Prompts - FALCON

In [196]:
import re
import pandas as pd

dfs = []
TOPICS = ['Indian climate', 'Indian defense', 'Indian economy', 'Indian infrastructure']
for topic in TOPICS:
  df = pd.read_csv(f'drive/MyDrive/DUCSResearch/Bias in LLMs/colab_notebooks/bias_llm/data/falcon-7b-200-google-scholar-17012024/{topic}/data.csv')
  dfs.append(df)

for df in dfs:
  df['Processed Text'] = df['Generated Text'].map(lambda x: x.lower())
  df['Processed Text'] = df['Processed Text'].map(lambda s: ' '.join([i for i in nltk.word_tokenize(s)[2:] if (i.isalpha() or ispunct(i)) and len(i) >= 3 and i not in stopwords_english]))

df_combined = pd.DataFrame()

for df in dfs:
  df_combined = df_combined.append(df, ignore_index=True)

df_combined

,Category,Prompt,Generated Text,i,MAX_LEN,Processed Text
0,Indian climate,Impact of climate change on Indian agriculture...,Impact of climate change on Indian agriculture...,0,198,climate change agriculture review abstractthe ...
1,Indian climate,"The predictive state: science, territory and t...","The predictive state: science, territory and t...",1,190,state science territory and the future the cli...
2,Indian climate,Water resources and climate change: An Indian ...,Water resources and climate change: An Indian ...,2,180,and climate change perspective the monsoon one...
3,Indian climate,Fractal dimensional analysis of Indian climati...,Fractal dimensional analysis of Indian climati...,3,198,analysis climatic dynamics institute tropical ...
4,Indian climate,Temperature and rainfall extremes change under...,Temperature and rainfall extremes change under...,4,189,rainfall extremes change under current and fut...
...,...,...,...,...,...,...
1978,Indian infrastructure,What Causes AgglomerationPolicy or Infrastruct...,What Causes AgglomerationPolicy or Infrastruct...,496,183,agglomerationpolicy infrastructure study organ...
1979,Indian infrastructure,The Scenario of FDI in Infrastructure of India,The Scenario of FDI in Infrastructure of India...,497,194,fdi infrastructure the infrastructure not suff...
1980,Indian infrastructure,The Impact Of Gaja Cyclone On Paddy And Rural ...,The Impact Of Gaja Cyclone On Paddy And Rural ...,498,198,gaja cyclone paddy and rural infrastructure th...
1981,Indian infrastructure,South-South investment in infrastructure: the ...,South-South investment in infrastructure: the ...,499,192,infrastructure the operation firms developing ...


In [197]:
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import AgglomerativeClustering
experiment='SVD/Agg'
umap_model = TruncatedSVD(n_components=100, random_state=42)
hdbscan_model = AgglomerativeClustering(n_clusters=4)
topic_model = get_topic_model(umap_model, hdbscan_model)

#### Indian climate

In [198]:
topic = 'Indian climate'

In [199]:
df = dfs[0]
df

,Category,Prompt,Generated Text,i,MAX_LEN,Processed Text
0,Indian climate,Impact of climate change on Indian agriculture...,Impact of climate change on Indian agriculture...,0,198,climate change agriculture review abstractthe ...
1,Indian climate,"The predictive state: science, territory and t...","The predictive state: science, territory and t...",1,190,state science territory and the future the cli...
2,Indian climate,Water resources and climate change: An Indian ...,Water resources and climate change: An Indian ...,2,180,and climate change perspective the monsoon one...
3,Indian climate,Fractal dimensional analysis of Indian climati...,Fractal dimensional analysis of Indian climati...,3,198,analysis climatic dynamics institute tropical ...
4,Indian climate,Temperature and rainfall extremes change under...,Temperature and rainfall extremes change under...,4,189,rainfall extremes change under current and fut...
...,...,...,...,...,...,...
496,Indian climate,Climate-forced sea-level lowstands in the Indi...,Climate-forced sea-level lowstands in the Indi...,496,197,lowstands the ocean during the last two millen...
497,Indian climate,Impact of ISO 9000 on organizational climate: ...,Impact of ISO 9000 on organizational climate: ...,497,182,iso organizational climate strategic change ma...
498,Indian climate,Examining mineral dust transport over the Indi...,Examining mineral dust transport over the Indi...,498,197,dust transport over the subcontinent using the...
499,Indian climate,Predicting the geographical distribution of an...,Predicting the geographical distribution of an...,499,184,geographical distribution invasive species chr...


In [200]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2024-01-27 08:03:52,866 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 08:03:52,939 - BERTopic - Dimensionality - Completed ✓
2024-01-27 08:03:52,944 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 08:03:52,968 - BERTopic - Cluster - Completed ✓
2024-01-27 08:03:52,977 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 08:03:53,601 - BERTopic - Representation - Completed ✓


In [201]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,0,146,0_climate_model_change_rainfall,"[climate, model, change, rainfall, summer, mon...","[monsoon, monsoons, monsoonal, climate, climat...","[climate, model, change, rainfall, summer, mon...",[droughts under the retrospective and projecte...
1,1,140,1_climate_change_energy_study,"[climate, change, energy, study, health, solar...","[climate, climatic, environmental, adaptation,...","[climate, change, energy, study, health, solar...",[and resilience climate change case study and ...
2,2,110,2_ocean_climate_variability_tropical,"[ocean, climate, variability, tropical, pacifi...","[monsoonal, climate, monsoons, monsoon, climat...","[ocean, climate, variability, tropical, pacifi...",[regional climate model for eastern africa and...
3,3,105,3_climate_change_agricultural_agriculture,"[climate, change, agricultural, agriculture, c...","[agriculture, climate, crops, agricultural, fa...","[climate, change, agricultural, agriculture, c...",[impacts climate change land productivity crop...


In [202]:
model_score = 0
for i in range(3):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 3

0.31178888888888884

#### Indian defence

In [203]:
topic = 'Indian defense'

In [204]:
df = dfs[1]

In [205]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2024-01-27 08:04:26,312 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 08:04:26,509 - BERTopic - Dimensionality - Completed ✓
2024-01-27 08:04:26,512 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 08:04:26,549 - BERTopic - Cluster - Completed ✓
2024-01-27 08:04:26,560 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 08:04:27,138 - BERTopic - Representation - Completed ✓


In [206]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,0,150,0_defence_study_research_british,"[defence, study, research, british, law, new, ...","[defence, defense, military, research, securit...","[defence, study, research, british, law, new, ...",[information seeking life scientists defence r...
1,1,135,1_defence_army_security_military,"[defence, army, security, military, budget, fo...","[military, procurement, army, defence, securit...","[defence, army, security, military, budget, fo...",[and pakistan pakistan the air force iaf and t...
2,2,120,2_cooperation_defence_security_strategic,"[cooperation, defence, security, strategic, nu...","[diplomacy, pakistan, asia, cooperation, vietn...","[cooperation, defence, security, strategic, nu...",[defence cooperation with other countries serv...
3,3,96,3_defence_sector_industry_manufacturing,"[defence, sector, industry, manufacturing, pol...","[procurement, fdi, manufacturing, industries, ...","[defence, sector, industry, manufacturing, pol...",[research and development and transitioning ab...


In [208]:
model_score = 0
for i in range(3):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 3

0.7455666666666666

#### Indian economy

In [209]:
topic = 'Indian economy'

In [210]:
df = dfs[2]

In [211]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

2024-01-27 08:04:56,021 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 08:04:56,200 - BERTopic - Dimensionality - Completed ✓
2024-01-27 08:04:56,203 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 08:04:56,244 - BERTopic - Cluster - Completed ✓
2024-01-27 08:04:56,261 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 08:04:57,615 - BERTopic - Representation - Completed ✓


In [212]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,0,219,0_economic_economy_growth_development,"[economic, economy, growth, development, count...","[economy, economic, gdp, reforms, growth, gove...","[economic, economy, growth, development, count...",[economic liberalization this paper examine th...
1,1,100,1_economy_sector_energy_agriculture,"[economy, sector, energy, agriculture, industr...","[agriculture, gdp, economy, agricultural, indu...","[economy, sector, energy, agriculture, industr...",[tertiary sector economy issues and implicatio...
2,2,83,2_impact_economy_tax_gst,"[impact, economy, tax, gst, inflation, economi...","[macroeconomic, gdp, economy, gst, economic, i...","[impact, economy, tax, gst, inflation, economi...",[economical impact gst economy the gst goods a...
3,3,78,3_foreign_financial_growth_economy,"[foreign, financial, growth, economy, investme...","[fdi, economy, gdp, investment, economic, inve...","[foreign, financial, growth, economy, investme...",[foreign direct investment fdi the economy for...


In [213]:
model_score = 0
for i in range(3):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 3

0.6303333333333333

#### Indian infra

In [214]:
topic = 'Indian infrastructure'

In [215]:
df = dfs[3]

In [216]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2024-01-27 08:05:34,386 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 08:05:34,656 - BERTopic - Dimensionality - Completed ✓
2024-01-27 08:05:34,669 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 08:05:34,758 - BERTopic - Cluster - Completed ✓
2024-01-27 08:05:34,773 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 08:05:35,945 - BERTopic - Representation - Completed ✓


In [217]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,0,164,0_infrastructure_study_research_public,"[infrastructure, study, research, public, mana...","[infrastructure, transportation, transport, co...","[infrastructure, study, research, public, mana...",[energy consumption impact planning and formul...
1,1,152,1_infrastructure_development_projects_investment,"[infrastructure, development, projects, invest...","[infrastructure, construction, governance, inv...","[infrastructure, development, projects, invest...",[emerging challenges this paper provides overv...
2,2,120,2_health_infrastructure_rural_urban,"[health, infrastructure, rural, urban, water, ...","[infrastructure, rural, healthcare, hospitals,...","[health, infrastructure, rural, urban, water, ...",[immunization coverage rural evidence from thi...
3,3,65,3_growth_infrastructure_economic_manufacturing,"[growth, infrastructure, economic, manufacturi...","[infrastructure, gdp, economy, econometric, gr...","[growth, infrastructure, economic, manufacturi...",[economic development and urbanization there a...


In [220]:
model_score = 0
for i in range(3):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 3

0.5617000000000001

#### Combined

In [222]:
topic = 'Combined'

In [223]:
df = df_combined
df

,Category,Prompt,Generated Text,i,MAX_LEN,Processed Text
0,Indian climate,Impact of climate change on Indian agriculture...,Impact of climate change on Indian agriculture...,0,198,climate change agriculture review abstractthe ...
1,Indian climate,"The predictive state: science, territory and t...","The predictive state: science, territory and t...",1,190,state science territory and the future the cli...
2,Indian climate,Water resources and climate change: An Indian ...,Water resources and climate change: An Indian ...,2,180,and climate change perspective the monsoon one...
3,Indian climate,Fractal dimensional analysis of Indian climati...,Fractal dimensional analysis of Indian climati...,3,198,analysis climatic dynamics institute tropical ...
4,Indian climate,Temperature and rainfall extremes change under...,Temperature and rainfall extremes change under...,4,189,rainfall extremes change under current and fut...
...,...,...,...,...,...,...
1978,Indian infrastructure,What Causes AgglomerationPolicy or Infrastruct...,What Causes AgglomerationPolicy or Infrastruct...,496,183,agglomerationpolicy infrastructure study organ...
1979,Indian infrastructure,The Scenario of FDI in Infrastructure of India,The Scenario of FDI in Infrastructure of India...,497,194,fdi infrastructure the infrastructure not suff...
1980,Indian infrastructure,The Impact Of Gaja Cyclone On Paddy And Rural ...,The Impact Of Gaja Cyclone On Paddy And Rural ...,498,198,gaja cyclone paddy and rural infrastructure th...
1981,Indian infrastructure,South-South investment in infrastructure: the ...,South-South investment in infrastructure: the ...,499,192,infrastructure the operation firms developing ...


In [224]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(df['Processed Text'], show_progress_bar=True)
topics, probs = topic_model.fit_transform(df['Processed Text'], embeddings)

Batches:   0%|          | 0/62 [00:00<?, ?it/s]

2024-01-27 08:07:26,412 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 08:07:26,593 - BERTopic - Dimensionality - Completed ✓
2024-01-27 08:07:26,595 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 08:07:26,843 - BERTopic - Cluster - Completed ✓
2024-01-27 08:07:26,849 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 08:07:27,700 - BERTopic - Representation - Completed ✓


In [225]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,0,872,0_economy_economic_growth_study,"[economy, economic, growth, study, impact, dev...","[economy, economic, gdp, reforms, employment, ...","[economy, economic, growth, study, impact, dev...",[economic liberalization this paper examine th...
1,1,414,1_defence_military_security_policy,"[defence, military, security, policy, defense,...","[defence, defense, military, procurement, secu...","[defence, military, security, policy, defense,...",[defence embedding dialogue and consultations ...
2,2,374,2_climate_change_variability_ocean,"[climate, change, variability, ocean, model, t...","[climate, monsoon, monsoons, climatic, agricul...","[climate, change, variability, ocean, model, t...",[and climate change perspective the monsoon on...
3,3,323,3_infrastructure_development_urban_public,"[infrastructure, development, urban, public, h...","[infrastructure, construction, economy, govern...","[infrastructure, development, urban, public, h...",[emerging challenges this paper provides overv...


In [226]:
model_score = 0
for i in range(4):
  docs = topic_model.get_representative_docs(i)
  score = 0
  for doc in docs:
    score += analyzer.polarity_scores(doc)['compound']
  score /= 3
  model_score += score
model_score / 4

0.8077333333333333